<a href="https://colab.research.google.com/github/isDaisyorDaisy/proejcts/blob/main/predicting_house_price_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library("dplyr")
library("data.table")
library("lubridate")
library("gbm")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last



Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loaded gbm 2.1.8



In [ ]:
st_d <- fread("data/sales_train.csv")
items_d <- fread("data/items.csv")
test_d <- fread("data/test.csv")

In [ ]:
sales_d <- merge(st_d, items_d[,c("item_id", "item_category_id")], by = "item_id", all.x = T)
test_d  <- merge(test_d, items_d[,c("item_id", "item_category_id")], by = "item_id", all.x = T)

# sales_d$date     <- as.Date(sales_d$date, "%d.%m.%Y")
# sales_d$day      <- as.factor(day(sales_d$date))
# sales_d$weekdays <- as.factor(weekdays(sales_d$date))
# sales_d$month    <- as.factor(month(sales_d$date))
# sales_d$year     <- as.factor(year(sales_d$date))

sales_d$icid <- as.factor(sales_d$item_category_id)
test_d$icid  <- as.factor(test_d$item_category_id)
head(sales_d)
head(test_d)

item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_category_id,icid
<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<fct>
0,01.09.2014,20,54,58,1,40,40
1,04.04.2014,15,55,4490,1,76,76
1,02.04.2014,15,55,4490,1,76,76
1,06.07.2014,18,55,4490,1,76,76
1,04.08.2014,19,55,4490,1,76,76
1,11.09.2014,20,55,4490,1,76,76


item_id,ID,shop_id,item_category_id,icid
<int>,<int>,<int>,<int>,<fct>
30,2587,5,40,40
30,7687,4,40,40
30,12787,6,40,40
30,17887,3,40,40
30,22987,2,40,40
30,28087,7,40,40


In [ ]:
do_p <- function(res, num){
    subm <- data.frame(ID = test_d$ID, item_cnt_month = res)
    write.csv(subm, file = paste("sub", num, ".csv", sep=""), row.names = F)
    return(subm)
}
getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}

## Try 1

score: 1.56324

In [ ]:
model <- lm(formula = item_cnt_day ~ shop_id + item_id + item_category_id, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "item_category_id")])
s1 <- do_p(res, 1)
head(s1)
# Score
# 1.56324

,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.213684
2,7687,1.214621
3,12787,1.212746
4,17887,1.215559
5,22987,1.216496
6,28087,1.211808


## Try 2

score: 1.52569

In [ ]:
model <- lm(formula = item_cnt_day ~ shop_id + item_id + icid, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "icid")])
s2 <- do_p(res, 2)
head(s2)
# Score
# 1.52569

,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.163262
2,7687,1.163587
3,12787,1.162938
4,17887,1.163912
5,22987,1.164237
6,28087,1.162613


In [ ]:
sales_d$date     <- as.Date(sales_d$date, "%d.%m.%Y")
sales_d$month    <- as.factor(month(sales_d$date))

In [ ]:
summary(sales_d)

    item_id           date            date_block_num     shop_id  
 Min.   :    0   Min.   :2013-01-01   Min.   : 0.00   Min.   : 0  
 1st Qu.: 4476   1st Qu.:2013-08-01   1st Qu.: 7.00   1st Qu.:22  
 Median : 9343   Median :2014-03-04   Median :14.00   Median :31  
 Mean   :10197   Mean   :2014-04-03   Mean   :14.57   Mean   :33  
 3rd Qu.:15684   3rd Qu.:2014-12-05   3rd Qu.:23.00   3rd Qu.:47  
 Max.   :22169   Max.   :2015-10-31   Max.   :33.00   Max.   :59  
                                                                  
   item_price        item_cnt_day      item_category_id      icid        
 Min.   :    -1.0   Min.   : -22.000   Min.   : 0       40     : 564652  
 1st Qu.:   249.0   1st Qu.:   1.000   1st Qu.:28       30     : 351591  
 Median :   399.0   Median :   1.000   Median :40       55     : 339585  
 Mean   :   890.9   Mean   :   1.243   Mean   :40       19     : 208219  
 3rd Qu.:   999.0   3rd Qu.:   1.000   3rd Qu.:55       37     : 192674  
 Max.   :307980.0   

In [ ]:
test_d$month <- 10
test_d$month <- as.factor(test_d$month)
test_d$date_block_num <- 34
head(test_d)

item_id,ID,shop_id,item_category_id,icid,month,date_block_num
<int>,<int>,<int>,<int>,<fct>,<fct>,<dbl>
30,2587,5,40,40,10,34
30,7687,4,40,40,10,34
30,12787,6,40,40,10,34
30,17887,3,40,40,10,34
30,22987,2,40,40,10,34
30,28087,7,40,40,10,34


## Try 3

score: 1.43737

In [ ]:
model <- lm(formula = log(item_cnt_day) ~ shop_id + item_id + icid + date_block_num, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "icid", "date_block_num")])

s3 <- do_p(exp(res), 3)
head(s3)

Warning message in log(item_cnt_day):
“NaNs produced”


,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.064008
2,7687,1.063994
3,12787,1.064023
4,17887,1.063980
5,22987,1.063965
6,28087,1.064037


In [ ]:
uids <- test_d$item_id %>% unique
u_price <- uids %>% lapply(., function(x){ sales_d[item_id == x]$item_price %>% getmode }) %>% unlist

test_d$item_price <- test_d$item_id %>% lapply(., function(x){u_price[uids == x]}) %>% unlist
test_d$item_price[is.na(test_d$item_price)] <- 0
head(test_d)

item_id,ID,shop_id,item_category_id,icid,month,date_block_num,item_price
<int>,<int>,<int>,<int>,<fct>,<fct>,<dbl>,<dbl>
30,2587,5,40,40,10,34,399
30,7687,4,40,40,10,34,399
30,12787,6,40,40,10,34,399
30,17887,3,40,40,10,34,399
30,22987,2,40,40,10,34,399
30,28087,7,40,40,10,34,399


## Try 4

score: 1.43841

In [ ]:
model <- lm(formula = log(item_cnt_day) ~ shop_id + item_id + icid + date_block_num + item_price, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "icid", "date_block_num", "item_price")])

s4 <- do_p(exp(res), 4)
head(s4)

Warning message in log(item_cnt_day):
“NaNs produced”


,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.063819
2,7687,1.063798
3,12787,1.063841
4,17887,1.063776
5,22987,1.063755
6,28087,1.063862


## Try 5

score: 1.44981

In [ ]:
model <- lm(formula = log(item_cnt_day) ~ shop_id + item_id + icid + date_block_num + item_price + month, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "icid", "date_block_num", "item_price", "month")])

s5 <- do_p(exp(res), 5)
head(s5)

Warning message in log(item_cnt_day):
“NaNs produced”


,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.085492
2,7687,1.085469
3,12787,1.085516
4,17887,1.085446
5,22987,1.085422
6,28087,1.085539


In [ ]:
sales_d$item_cnt_day_2[sales_d$item_cnt_day <= 0] <- 0.1

## Try 6

score: 1.44631

In [ ]:
model <- lm(formula = log(item_cnt_day_2) ~ shop_id + item_id + icid + date_block_num + item_price + month, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "icid", "date_block_num", "item_price", "month")])

s6 <- do_p(exp(res), 6)
head(s6)

,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.081754
2,7687,1.081736
3,12787,1.081771
4,17887,1.081718
5,22987,1.081700
6,28087,1.081789


## Try 7

score: 1.33841

In [ ]:
sales_d$item_cnt_m <- sales_d$item_cnt_day / 12 + 1

model <- lm(formula = log(item_cnt_m) ~ shop_id + icid + date_block_num + item_price + month, data = sales_d)
res <- predict(model, test_d[, c("shop_id", "item_id", "icid", "date_block_num", "item_price", "month")])

s7<- do_p((exp(res) - 1) * 12, 7)
head(s7)

,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.120843
2,7687,1.120892
3,12787,1.120793
4,17887,1.120941
5,22987,1.120991
6,28087,1.120744


## Try 8

score: 2.13690

In [ ]:
model <- gbm(item_cnt_day ~ shop_id + item_id + month + date_block_num + item_category_id,
             data = sales_d,
             shrinkage = 0.01,
             distribution = "gaussian",
             n.trees = 1000,
             interaction.depth = 5, 
             bag.fraction = 0.5,
             train.fraction = 0.8,
             verbose = F)

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        6.0239         10.1985     0.0100    0.0009
     2        6.0231         10.1985     0.0100    0.0008
     3        6.0213         10.1984     0.0100    0.0016
     4        6.0205         10.1985     0.0100    0.0008
     5        6.0193         10.1986     0.0100    0.0007
     6        6.0181         10.1986     0.0100    0.0008
     7        6.0173         10.1988     0.0100    0.0007
     8        6.0161         10.1989     0.0100    0.0008
     9        6.0152         10.1989     0.0100    0.0010
    10        6.0134         10.1990     0.0100    0.0015
    20        6.0037         10.2009     0.0100    0.0006
    40        5.9836         10.2055     0.0100    0.0007
    60        5.9643         10.2098     0.0100    0.0010
    80        5.9478         10.2147     0.0100    0.0006
   100        5.9282         10.2169     0.0100    0.0009
   120        5.9115         10.2179     0.0100    0.0005
   140        

In [ ]:
res <- predict(model, 
               newdata = test_d[,c("shop_id","item_id","month","date_block_num","item_category_id")], 
               n.trees = 1000)
s8 <- do_p(res, 8)
head(s8)

,ID,item_cnt_month
,<int>,<dbl>
1,2587,1.060889
2,7687,1.060889
3,12787,1.094212
4,17887,1.060889
5,22987,1.060889
6,28087,1.094212
